In [1]:
import pandas as pd
import PIL
import numpy as np
import cv2
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pathlib
from sklearn.model_selection import train_test_split
import cv2 as cv
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import mlflow as mf

In [2]:

# Labeling function to convert path into Windows Path:
def Labeller(path):
    
    path = pathlib.Path(path)
    path=(list(path.glob("*.jpg")))
    return path


In [3]:
# cancer type stored in each folder's path 
Glioma="C:/Users/chakr/OneDrive/Desktop/Projects/Brain Tumor Detection End to End ML Project/Train/Glioma/images"
Meningioma="C:/Users/chakr/OneDrive/Desktop/Projects/Brain Tumor Detection End to End ML Project/Train/Meningioma/images"
No_Tumor ="C:/Users/chakr/OneDrive/Desktop/Projects/Brain Tumor Detection End to End ML Project/Train/No Tumor/images"
Pituitary="C:/Users/chakr/OneDrive/Desktop/Projects/Brain Tumor Detection End to End ML Project/Train/Pituitary/images"
cancer_types = ["Glioma","Meningioma","No Tumor","Pituitary"]

# Apllying Labeller
Glioma  = Labeller(Glioma)
Meningioma  = Labeller(Meningioma)
No_Tumor  = Labeller(No_Tumor)
Pituitary  = Labeller(Pituitary)
Cancer_type_list =[Glioma,Meningioma,No_Tumor,Pituitary]
PIL.Image.open(Glioma[50])

# Label Dictionary for each type
Cancer_label = {
    "Glioma":0,
    "Meningioma": 1,
    "No_Tumor":2,
    "Pituitary":3}

In [4]:
# creating x and y by creating list with image array and image labels by zip method
x = []
y = []
for cancer_type, label in zip(Cancer_type_list, Cancer_label.values()):
    for j in cancer_type:
        vector = cv.imread(str(j))
        vector = cv.resize(vector, (180, 180))
        x.append(vector)
        y.append(label)

In [5]:

x = np.array(x)
y = np.array(y)
x.shape
#
# Splitting of data
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)

x_train_scaled = x_train/255.0
x_test_scaled = x_test/255.0

In [6]:
# data augmentation : here I rotated 20%, zoomed 20% of my total images and added usesd that in my model
data_augmentation = tf.keras.Sequential(
  [
    tf.keras.layers.RandomFlip("horizontal",input_shape=(180, 180,3)),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2)
#     tf.image.stateless_random_brightness(0.1)

      
  ]
)

C:\Users\chakr\anaconda3\Lib\site-packages\keras\src\layers\preprocessing\tf_data_layer.py:19: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [7]:
# CNN_model
num_classes = 4

CNN_model = tf.keras.Sequential([
  data_augmentation,
    tf.keras.layers.Conv2D(kernel_size=(4, 4), padding='same', activation='relu',strides=(2,2),filters=25),
#   tf.keras.layers.MaxPooling2D(),
   tf.keras.layers.Conv2D(kernel_size=(4, 4), padding='same', activation='relu',strides=(2,2),filters=25),
#    tf.keras.layers.MaxPooling2D(),
  
   tf.keras.layers.Conv2D(kernel_size=(4, 4), padding='same', activation='relu',strides=(2,2),filters=25),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(16, 3, padding='same', activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
   tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(12, activation='relu'),
  tf.keras.layers.Dense(num_classes, activation = "softmax")
])
# Compiling the model with Adam optimizer and sparse categorical crossentropy loss
CNN_model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
# fitting          
history=CNN_model.fit(x_train_scaled, y_train, epochs=24,validation_split=0.2)

Epoch 1/24


C:\Users\chakr\anaconda3\Lib\site-packages\keras\src\backend\tensorflow\nn.py:635: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


89/89 ━━━━━━━━━━━━━━━━━━━━ 31s 241ms/step - accuracy: 0.3100 - loss: 1.3709 - val_accuracy: 0.2911 - val_loss: 1.3560
Epoch 2/24
89/89 ━━━━━━━━━━━━━━━━━━━━ 19s 208ms/step - accuracy: 0.3480 - loss: 1.2940 - val_accuracy: 0.4993 - val_loss: 1.0585
Epoch 3/24
89/89 ━━━━━━━━━━━━━━━━━━━━ 19s 208ms/step - accuracy: 0.5250 - loss: 1.1098 - val_accuracy: 0.6540 - val_loss: 0.8818
Epoch 4/24
89/89 ━━━━━━━━━━━━━━━━━━━━ 18s 204ms/step - accuracy: 0.5917 - loss: 0.9755 - val_accuracy: 0.6821 - val_loss: 0.8349
Epoch 5/24
89/89 ━━━━━━━━━━━━━━━━━━━━ 19s 212ms/step - accuracy: 0.6149 - loss: 0.9347 - val_accuracy: 0.6948 - val_loss: 0.8033
Epoch 6/24
89/89 ━━━━━━━━━━━━━━━━━━━━ 19s 217ms/step - accuracy: 0.6428 - loss: 0.8724 - val_accuracy: 0.7229 - val_loss: 0.7519
Epoch 7/24
89/89 ━━━━━━━━━━━━━━━━━━━━ 18s 195ms/step - accuracy: 0.6569 - loss: 0.8539 - val_accuracy: 0.6273 - val_loss: 0.8347
Epoch 8/24
89/89 ━━━━━━━━━━━━━━━━━━━━ 19s 209ms/step - accuracy: 0.6658 - loss: 0.8203 - val_accuracy: 0.737

In [8]:
from sklearn.metrics import classification_report
import numpy as np

# predictions for the test or validation data
y_pred_prob = CNN_model.predict(x_test_scaled)  # Predict probabilities
y_pred = np.argmax(y_pred_prob, axis=1)         # Convert to predicted class labels

# Generating classification report
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=['Glioma', 'Meningioma', 'No_Tumor', 'Pituitary']))


38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 47ms/step
Classification Report:
              precision    recall  f1-score   support

      Glioma       0.63      0.77      0.69       287
  Meningioma       0.66      0.71      0.69       349
    No_Tumor       0.92      0.66      0.77       182
   Pituitary       0.90      0.80      0.85       367

    accuracy                           0.75      1185
   macro avg       0.78      0.74      0.75      1185
weighted avg       0.77      0.75      0.75      1185



In [9]:
# Saved model for use
save_model(CNN_model,"C:/Users/chakr/OneDrive/Desktop/Projects/Brain Tumor Detection End to End ML Project/model.keras")

In [10]:
# loaded model after saving it in local
loaded_model=load_model("C:/Users/chakr/OneDrive/Desktop/Projects/Brain Tumor Detection End to End ML Project/model.keras")

In [11]:
###   Machine Learning Engineer  ###
# This section initializes MLflow for tracking the brain cancer detection project.
# It sets up the experiment, logs model parameters and metrics, and saves the trained model.

In [25]:
# Setting up experiment. Can be done from mlflow on localhost as well
# write mlflow ui on git bash to open mlflow locally 
mf.set_experiment("Brain Cancer Detection")


2024/09/09 20:15:09 INFO mlflow.tracking.fluent: Experiment with name 'Brain Cancer Detection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/776d24ce681b4c33b9f5825c9b09a105', creation_time=1725893107729, experiment_id='0', last_update_time=1725893107729, lifecycle_stage='active', name='Brain Cancer Detection', tags={}>

In [ ]:
# before going forward. create a repo with this notebook and the saved_model, then continue 
# create a dagshub account and connect it with github

In [26]:
# setting env 
import os
os.environ["MLFLOW_TRACKING_USERNAME"] = "Gourab79"
os.environ["MLFLOW_TRACKING_PASSWORD"]= "2ea2fe15c57225c20b878205648ef203d363cbf6"
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Gourab79/Brain-Cancer-Detector-End-to-End-ML-Project.mlflow"

In [27]:
# adding metrics and params to mlflow 
import mlflow as mf
import dagshub
dagshub.init(repo_owner='Gourab79', repo_name='Brain-Cancer-Detector-End-to-End-ML-Project', mlflow=True)
# Start MLflow run
with mf.start_run():

    # Log model parameters
    mf.log_param("num_classes", 4)
    mf.log_param("conv_layers", [
        {"kernel_size": (4, 4), "padding": "same", "activation": "relu", "strides": (2,2), "filters": 25},
        {"kernel_size": (4, 4), "padding": "same", "activation": "relu", "strides": (2,2), "filters": 25},
        {"kernel_size": (4, 4), "padding": "same", "activation": "relu", "strides": (2,2), "filters": 25},
        {"kernel_size": 3, "padding": "same", "activation": "relu", "filters": 16},
        {"kernel_size": 3, "padding": "same", "activation": "relu", "filters": 16},
        {"kernel_size": 3, "padding": "same", "activation": "relu", "filters": 16},
    ])
    mf.log_param("pooling_layers", [
        {"pool_size": (2, 2)},
        {"pool_size": (2, 2)},
        {"pool_size": (2, 2)},
        {"pool_size": (2, 2)},
    ])
    mf.log_param("dropout_rate", 0.2)
    mf.log_param("dense_layers", [
        {"units": 128, "activation": "relu"},
        {"units": 64, "activation": "relu"},
        {"units": 12, "activation": "relu"},
        {"units": 4, "activation": "softmax"},
    ])
    mf.log_param("optimizer", "adam")
    mf.log_param("loss_function", "SparseCategoricalCrossentropy")
    mf.log_param("metrics", "accuracy")
    mf.log_param("epochs", 24)
    mf.log_param("validation_split", 0.2)

    # Log classification report metrics
    mf.log_metric("Glioma_precision", 0.76)
    mf.log_metric("Glioma_recall", 0.66)
    mf.log_metric("Glioma_f1_score", 0.71)

    mf.log_metric("Meningioma_precision", 0.70)
    mf.log_metric("Meningioma_recall", 0.75)
    mf.log_metric("Meningioma_f1_score", 0.72)

    mf.log_metric("No_Tumor_precision", 0.82)
    mf.log_metric("No_Tumor_recall", 0.86)
    mf.log_metric("No_Tumor_f1_score", 0.84)

    mf.log_metric("Pituitary_precision", 0.90)
    mf.log_metric("Pituitary_recall", 0.90)
    mf.log_metric("Pituitary_f1_score", 0.90)
    mf.sklearn.log_model(CNN_model, "CNN")


Initialized MLflow to track repo "Gourab79/Brain-Cancer-Detector-End-to-End-ML-Project"

Repository Gourab79/Brain-Cancer-Detector-End-to-End-ML-Project initialized!

2024/09/09 20:15:39 WARNING mlflow.models.model: Input example should be provided to infer model signature if the model signature is not provided when logging the model.
2024/09/09 20:15:40 INFO mlflow.tracking._tracking_service.client: 🏃 View run luxuriant-moose-235 at: https://dagshub.com/Gourab79/Brain-Cancer-Detector-End-to-End-ML-Project.mlflow/#/experiments/0/runs/55f4f65d56c5404393ca27bec6d728d1.
2024/09/09 20:15:40 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/Gourab79/Brain-Cancer-Detector-End-to-End-ML-Project.mlflow/#/experiments/0.


In [28]:
import mlflow
logged_model = 'runs:/55f4f65d56c5404393ca27bec6d728d1/CNN'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(x_test_scaled)

38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 53ms/step


array([[0.2379717 , 0.7105963 , 0.02962596, 0.02180606],
       [0.04342366, 0.01827282, 0.01424058, 0.9240629 ],
       [0.3514632 , 0.43090218, 0.00938431, 0.20825034],
       ...,
       [0.02041199, 0.01552758, 0.00567913, 0.9583813 ],
       [0.3907071 , 0.15467419, 0.0125036 , 0.44211507],
       [0.38833782, 0.55721843, 0.01051051, 0.04393319]], dtype=float32)

In [42]:
loaded_model

mlflow.pyfunc.loaded_model:
  artifact_path: CNN
  flavor: mlflow.sklearn
  run_id: 55f4f65d56c5404393ca27bec6d728d1